<a href="https://colab.research.google.com/github/zuzannazak/PhD2022Bialowieza/blob/main/%F0%9F%92%96projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#💖Projekt z ekonometrii💖

**Stworzony przez:**
Wiktoria Rakowska

Monika Wesołowska

Zuzanna Żak


**Dane:**
*   https://github.com/pbiecek/SmarterPoland_blog/tree/master/dane/Bialowieza/